# Generate PARAMETERIZED Synthetic Data

In [1]:
# imports

import trimesh
import os
import sys
import subprocess
import torch
import numpy as np
os.environ["GEOMSTATS_BACKEND"] = "pytorch"
import geomstats.backend as gs
import my28brains.discrete_surfaces



print(os.environ["GEOMSTATS_BACKEND"])


INFO: Using pytorch backend


pytorch


In [2]:
gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
work_dir = os.getcwd()

my28brains_dir = os.path.join(work_dir, "my28brains")

sys_dir = os.path.dirname(work_dir)
sys.path.append(sys_dir)
sys.path.append(my28brains_dir)

# this forces the notebook to re-load
import importlib
importlib.reload(my28brains.discrete_surfaces)

from my28brains.discrete_surfaces import (
    DiscreteSurfaces,
    ElasticMetric,
)

import my28brains.default_config as default_config

print(os.environ["GEOMSTATS_BACKEND"])

pytorch


# Generate Source/Target meshes

## 1. Sphere Mesh

In [3]:
sphere = trimesh.creation.icosphere(subdivisions=3, radius=30.0)

# sphere.show()

## 1 Ellipsoid Mesh

In [4]:
# Create a scaling matrix for the semi-axes lengths
scales = np.array([2, 2, 3])
scale_matrix = np.diag(scales)

scale_matrix = gs.array(scale_matrix)

# Apply the scaling transformation to the mesh vertices
scaled_vertices = sphere.vertices.dot(scale_matrix)

# Create a new mesh with the scaled vertices
ellipsoid = trimesh.Trimesh(vertices=scaled_vertices, faces=sphere.faces)
# ellipsoid.show()

In [5]:
print(os.environ["GEOMSTATS_BACKEND"])

pytorch


# Define surface space

In [6]:
SURFACE_SPACE = DiscreteSurfaces(faces = sphere.faces)

METRIC = ElasticMetric(
    space = SURFACE_SPACE,
    a0=default_config.a0,
    a1=default_config.a1,
    b1=default_config.b1,
    c1=default_config.c1,
    d1=default_config.d1,
    a2=default_config.a2,)

print(sphere.faces.shape)
print(ellipsoid.faces.shape)

# METRIC = ElasticMetric(
#     space = SURFACE_SPACE,
#     a0=default_config.a0,
#     a1=0,
#     b1=0,
#     c1=default_config.c1,
#     d1=0,
#     a2=default_config.a2,)

print(os.environ["GEOMSTATS_BACKEND"])

(1280, 3)
(1280, 3)
pytorch


# Remove degenerate faces of these meshes

In [7]:
# area_threshold = 0.0005

# face_areas = SURFACE_SPACE.face_areas(torch.tensor(sphere.vertices))
# face_mask = ~gs.less(face_areas, area_threshold)
# sphere.update_faces(face_mask)
# print(sphere.faces.shape)

# face_areas = SURFACE_SPACE.face_areas(torch.tensor(ellipsoid.vertices))
# face_mask = ~gs.less(face_areas, area_threshold)
# ellipsoid.update_faces(face_mask)
# print(ellipsoid.faces.shape)

# SURFACE_SPACE = DiscreteSurfaces(faces = sphere.faces)

# METRIC = ElasticMetric(
#     space = SURFACE_SPACE,
#     a0=default_config.a0,
#     a1=default_config.a1,
#     b1=default_config.b1,
#     c1=default_config.c1,
#     d1=default_config.d1,
#     a2=default_config.a2,)

# Geodesic Regression

In [8]:
n_vertices = sphere.vertices.shape[0]
print(n_vertices)

def geodesics_sphere_to_ellipsoid(
    n_geodesics=1, n_times=5
):
    """Generate a dataset of geodesics that transform spheres into ellipsoids."""
    dim = 3
    
    geodesics = gs.zeros((n_geodesics, n_times, n_vertices, dim))
    times = gs.arange(0, 1, 1 / n_times)
    for i_geodesic in range(n_geodesics):
        start_surface = gs.array(sphere.vertices)
        end_surface = gs.array(ellipsoid.vertices)
        print(start_surface.shape)
        geodesic = METRIC.geodesic(initial_point = start_surface, end_point = end_surface)
        geodesics[i_geodesic] = geodesic(times)

    return geodesics

642


In [9]:
print(os.environ["GEOMSTATS_BACKEND"])

pytorch


In [10]:
geodesics_sphere_to_ellipsoid()

torch.Size([642, 3])
geodesic shapetorch.Size([3, 642, 3])
flattened geodesic shapetorch.Size([5778])
2D point arraytorch.Size([642, 3])
face_areas (642, 3)
shape of point in surface_one_forms: torch.Size([642, 3])
surface_one_forms: torch.Size([1280, 2, 3])
face_areas determinant: tensor([30.6237, 30.6379, 30.5502,  ..., 39.0885, 39.0489, 39.6171])
2D point arraytorch.Size([642, 3])
face_areas (642, 3)
shape of point in surface_one_forms: torch.Size([642, 3])
surface_one_forms: torch.Size([1280, 2, 3])
face_areas determinant: tensor([48.9766, 48.9356, 48.7343,  ..., 62.5415, 62.4508, 63.3874])
2D point arraytorch.Size([642, 3])
face_areas (642, 3)
shape of point in surface_one_forms: torch.Size([642, 3])
surface_one_forms: torch.Size([1280, 2, 3])
face_areas determinant: tensor([71.4071, 71.2962, 70.9536,  ..., 91.2064, 91.0521, 92.4399])
nondegenerate midpoint shapetorch.Size([3, 642, 3])
midpoint shapetorch.Size([3, 642, 3])
3
2D point arraytorch.Size([642, 3])
face_areas (642, 3)
s

RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [1280, 3] but got: [1280, 2].